In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from collections import Counter

from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.utils import resample
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from scipy.stats import mannwhitneyu, entropy, kstest

In [2]:
import os

os.environ['OMP_NUM_THREADS'] = '4'
os.environ['OPENBLAS_NUM_THREADS'] = '4'
os.environ['NUM_WORKERS'] = '4'
os.environ['MKL_NUM_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '4'

In [4]:
cd /media/HlabShare/sleep_dcc_yifan/final_production/Source\ Data

/media/HlabShare/sleep_dcc_yifan/final_production/Source Data


In [5]:
df = pd.read_excel('Source data Fig 2.xlsx',engine='openpyxl')
df = df[(df['perc_sleep']<=0.33)|(df['perc_sleep']>=0.66)] #filter line 
df.head()

,Animal,perc_sleep,DCC,T4,T3,T2,T1,Behavior,cat1,cat2,CV,FR,ZT,SWA,Alpha,Beta
1,XYF06,0.000000,0.119316,0.733333,0.313333,0.583333,0.507778,0.545556,0,0,1.299105,0.960447,13,0.757911,1.911381,2.355457
2,XYF06,0.778889,0.226115,0.583333,0.507778,0.000000,0.417778,0.208889,1,3,1.331130,1.097994,15,0.971613,1.958712,2.345246
3,XYF06,0.176667,0.180043,0.000000,0.417778,0.778889,0.914444,0.846667,0,0,1.274289,0.792273,17,0.910909,2.355083,2.760575
4,XYF06,0.813333,0.107884,0.778889,0.914444,0.176667,0.381111,0.278889,1,3,1.318826,0.987282,19,0.752604,2.122211,2.614125
5,XYF06,0.317778,0.192556,0.176667,0.381111,0.813333,0.856667,0.835000,0,1,1.266260,0.911802,21,0.962503,2.229426,2.794184


# Basic Functions

In [8]:
def train_test_splits(data_df,class_balance=True,class_col='cat1',test_size=0.15,n_splits=10):
    if class_balance == True:
        data_df = data_df.groupby(class_col)
        data_df = data_df.apply(lambda x: x.sample(data_df.size().min(), random_state=42).reset_index(drop=True))
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)
    split_idxs = sss.split(data_df, data_df[class_col])
    return data_df, split_idxs

def train_val_test_splits(data_df,class_balance=True,class_col='cat1',test_size=0.1,val_size=0.1,n_splits=10,random_state=42):
    if class_balance == True:
        data_df = data_df.groupby(class_col)
        data_df = data_df.apply(lambda x: x.sample(data_df.size().min(), random_state=random_state).reset_index(drop=True))
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    split_idxs = sss.split(data_df, data_df[class_col])
    train_idxs, test_idxs, val_idxs = [],[],[]
    for train,test in split_idxs:
        test_idxs.append(test)
        train_df = data_df.iloc[train]
        train_labels = data_df.iloc[train][class_col]
        if val_size == test_size:
            sss2 = StratifiedShuffleSplit(n_splits=1, test_size=len(test)/len(train), random_state=42)
            split_idxs2 = sss2.split(train_df,train_labels)
            for train2,val in split_idxs2:
                train_idxs.append(train[train2])
                val_idxs.append(train[val])
        else:
            raise NotImplementedError
    split_idxs = zip(train_idxs,val_idxs,test_idxs)
    return data_df, split_idxs


def train_classifiers(data_df, model, split_idxs, model_kwargs = None, feature_cols=['T1','T2','T3','T4','CT'], class_col='cat1'):
    models = []
    for train_idxs, test_idxs in split_idxs:
        if model == 'random_forest':
            if model_kwargs is None:
                iter_model = RandomForestClassifier(random_state=42)
            else:
                iter_model = RandomForestClassifier(random_state=42,**model_kwargs)
        elif model == 'logistic_regression':
            if model_kwargs is None:
                iter_model = LogisticRegression(random_state=69)
            else:
                iter_model = LogisticRegression(random_state=69,**model_kwargs)
        elif model == 'xgboost':
            if model_kwargs is None:
                iter_model = XGBClassifier(random_state=42)
            else:
                iter_model = XGBClassifier(random_state=42,**model_kwargs)
        x = data_df.iloc[train_idxs][feature_cols]
        y = data_df.iloc[train_idxs][class_col]
        iter_model.fit(x,y)
        models.append(iter_model)
    return models

def eval_classifiers(data_df, models, split_idxs, evaluator, eval_set='test',feature_cols=['T1','T2','T3','T4','CT'], class_col='cat1'):
    metrics = []
    for iter_,(train_idxs, test_idxs) in enumerate(split_idxs):
        model = models[iter_]
        if eval_set == 'test':
            x = data_df.iloc[test_idxs][feature_cols]
            y = data_df.iloc[test_idxs][class_col]
        elif eval_set == 'train':
            x = data_df.iloc[train_idxs][feature_cols]
            y = data_df.iloc[train_idxs][class_col]
        y_ = model.predict(x)
        metric = evaluator(y,y_)
        metrics.append(metric)
    return metrics

# Basic Models (Logistic Regression)

In [16]:

model = 'logistic_regression'

evaluator = accuracy_score
feature_cols_lol = [['Behavior'],['ZT'],['FR'],['CV'],['SWA'],['DCC']]

class_col = 'cat1'
n_splits= 100
test_size = 0.22
class_balance = True
random_state = 42

model_kwargs = None
train_metrics_means, test_metrics_means = [],[]
test_results_by_feature_dict = {}

test_results = []

for feature_cols in feature_cols_lol:
    data_df, split_idxs = train_test_splits(df, class_balance=class_balance, class_col=class_col, test_size=test_size, n_splits=n_splits)
    classifiers = train_classifiers(data_df, model, split_idxs, model_kwargs, feature_cols=feature_cols, class_col=class_col)
    
    data_df, split_idxs = train_test_splits(df, class_balance=class_balance, class_col=class_col, test_size=test_size, n_splits=n_splits)
    train_metrics = eval_classifiers(data_df, classifiers, split_idxs, evaluator, eval_set='train',feature_cols=feature_cols, class_col=class_col)
    
    data_df, split_idxs = train_test_splits(df, class_balance=class_balance, class_col=class_col, test_size=test_size, n_splits=n_splits)
    test_metrics = eval_classifiers(data_df, classifiers, split_idxs, evaluator, eval_set='test',feature_cols=feature_cols, class_col=class_col)
    #print(np.mean(train_metrics),"+\-",stats.sem(train_metrics),np.mean(test_metrics),"+\-",stats.sem(test_metrics))
    test_results_by_feature_dict[feature_cols[0]] = [np.mean(train_metrics),stats.sem(test_metrics),np.mean(test_metrics),stats.sem(test_metrics)]
    test_results.append(test_metrics)

In [17]:
df_result = pd.DataFrame({'behavior':test_results[0],
                   'ZT':test_results[1],'FR':test_results[2],'CV':test_results[3],
                   'SWA':test_results[4],'DCC':test_results[5]})

In [ ]:
df_result.describe()  

In [ ]:
sns.boxplot(data=df_result)

### Hyperparameter Tuning (XGBoost)

In [20]:
def train_val_test_splits(data_df,class_balance=True,class_col='cat1',test_size=0.1,val_size=0.1,n_splits=10,random_state=42):
    if class_balance == True:
        data_df = data_df.groupby(class_col)
        data_df = data_df.apply(lambda x: x.sample(data_df.size().min(), random_state=random_state).reset_index(drop=True))
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    split_idxs = sss.split(data_df, data_df[class_col])
    train_idxs, test_idxs, val_idxs = [],[],[]
    for train,test in split_idxs:
        test_idxs.append(test)
        train_df = data_df.iloc[train]
        train_labels = data_df.iloc[train][class_col]
        if val_size == test_size:
            sss2 = StratifiedShuffleSplit(n_splits=1, test_size=len(test)/len(train), random_state=42)
            split_idxs2 = sss2.split(train_df,train_labels)
            for train2,val in split_idxs2:
                train_idxs.append(train[train2])
                val_idxs.append(train[val])
        else:
            raise NotImplementedError
    split_idxs = zip(train_idxs,val_idxs,test_idxs)
    return data_df, split_idxs


def train_classifiers(data_df, model, split_idxs, model_kwargs = None, feature_cols=['T1','T2','T3','T4','CT'], class_col='cat1'):
    models = []
    for train_idxs, val_idxs, test_idxs in split_idxs:
        if model == 'random_forest':
            if model_kwargs is None:
                iter_model = RandomForestClassifier(random_state=42)
            else:
                iter_model = RandomForestClassifier(random_state=42,**model_kwargs)
        elif model == 'logistic_regression':
            if model_kwargs is None:
                iter_model = LogisticRegression(random_state=69)
            else:
                iter_model = LogisticRegression(random_state=69,**model_kwargs)
        elif model == 'xgboost':
            if model_kwargs is None:
                iter_model = XGBClassifier(random_state=42)
            else:
                iter_model = XGBClassifier(random_state=42,**model_kwargs)
        x = data_df.iloc[train_idxs][feature_cols]
        y = data_df.iloc[train_idxs][class_col]
        iter_model.fit(x,y)
        models.append(iter_model)
    return models

def eval_classifiers(data_df, models, split_idxs, evaluator, eval_set='test',feature_cols=['T1','T2','T3','T4','CT'], class_col='cat1'):
    metrics = []
    for iter_,(train_idxs, val_idxs, test_idxs) in enumerate(split_idxs):
        model = models[iter_]
        if eval_set == 'test':
            x = data_df.iloc[test_idxs][feature_cols]
            y = data_df.iloc[test_idxs][class_col]
        elif eval_set == 'train':
            x = data_df.iloc[train_idxs][feature_cols]
            y = data_df.iloc[train_idxs][class_col]
        y_ = model.predict(x)
        metric = evaluator(y,y_)
        metrics.append(metric)
    return metrics

In [21]:
def objective(space):
    data_df,split_idxs = train_val_test_splits(df,class_balance=class_balance,class_col=class_col,test_size=0.1,n_splits=n_splits)
    val_metrics, test_metrics = [],[]
    for train_idxs, val_idxs, test_idxs in split_idxs:
        iter_model = XGBClassifier(
        learning_rate = space['learning_rate'],
        n_estimators = int(space['n_estimators']),
        max_depth = int(space['max_depth']),
        #gamma = space['gamma'],
        reg_alpha = space['reg_alpha'],
        reg_lambda = space['reg_lambda'],
        random_state = int(space['random_state'])
        )
        x = data_df.iloc[train_idxs][feature_cols]
        y = data_df.iloc[train_idxs][class_col]
        iter_model.fit(x,y)
        
        x2 = data_df.iloc[val_idxs][feature_cols]
        y2 = data_df.iloc[val_idxs][class_col]
        y2_ = iter_model.predict(x2)
        val_metric = evaluator(y2,y2_) #evaluator defined in global scope
        val_metrics.append(val_metric)

        x3 = data_df.iloc[test_idxs][feature_cols]
        y3 = data_df.iloc[test_idxs][class_col]
        y3_ = iter_model.predict(x3)
        test_metric = evaluator(y3,y3_)
        test_metrics.append(test_metric)

    metric = -1 * np.mean(val_metrics)
    #print('SCORES:', np.mean(val_metrics), np.mean(test_metrics))
    return {'loss': metric, 'status': STATUS_OK}


In [ ]:
model = 'xgboost'
evaluator = balanced_accuracy_score
feature_cols =['Behavior','ZT']
class_col = 'cat1'
n_splits= 20
test_size = 0.1
class_balance = False


space={'learning_rate': hp.uniform('learning_rate',1E-1,1E4),
      'n_estimators': hp.quniform('n_estimators',1,1000,5),
       'max_depth': hp.quniform('max_depth',1,10,1),
       'gamma': hp.quniform('gamma',0,0.1,0.01),
       'reg_alpha': hp.uniform('reg_alpha',0,20),
       'reg_lambda': hp.uniform('reg_lambda',0,100000000),
       'random_state': 42 
      }

trials = Trials()
base_best_hyperparams = fmin(fn = objective,
                       space = space,
                       algo = tpe.suggest,
                       max_evals = 1000,
                       trials = trials)
print(base_best_hyperparams)


# use hyperparams to run

In [ ]:
model = 'xgboost'
evaluator = balanced_accuracy_score
class_col = 'cat1'
n_splits= 200
test_size = 0.2
class_balance = False

#OPTIMAL HYPERPARAMETERS

base_hps = {#'gamma': 0.05,
 'learning_rate': 3687.1577557642274,
 'max_depth': int(2.0),
 'n_estimators': int(200.0),
 'reg_alpha': 0.644242525839416312,
 'reg_lambda': 1990899.2470538504}

fr_hps = {#'gamma': 0.01,
 'learning_rate': 3991.228385329264,
 'max_depth': int(2.0),
 'n_estimators': int(435.0),
 'reg_alpha': 1.967722248935879,
 'reg_lambda': 137488.76528007258}

cv_hps = {#'gamma': 0.07,
 'learning_rate': 7398.910949037821,
 'max_depth': int(3.0),
 'n_estimators': int(500.0),
 'reg_alpha': 1.5460545805746793,
 'reg_lambda': 220523.1005840188}
    
dcc_hps = {# 'gamma': 0.1,
 'learning_rate': 3549.976010895837,
 'max_depth': int(2.0),
 'n_estimators': int(400.0),
 'reg_alpha': 1.11745609628605068,
 'reg_lambda': 1955442.3496703221}

all_hps = {#'gamma': 0.1,
 'learning_rate': 4195.186967321277,
 'max_depth': int(2.0),
 'n_estimators': int(530.0),
 'reg_alpha': 0.3924498620840177,
 'reg_lambda': 1819330.6569741974}

hp_dicts_list = [dcc_hps]

feature_cols_list = [['DCC','Behavior','ZT']]


for hps,feature_cols in zip(hp_dicts_list,feature_cols_list):
    
    # data_df,split_idxs = train_val_test_splits(df,class_balance=class_balance,class_col=class_col,test_size=test_size,val_size=test_size,n_splits=n_splits)
    data_df, split_idxs = train_test_splits(df, class_balance=class_balance, class_col=class_col, test_size=test_size, n_splits=n_splits)
    classifiers = train_classifiers(data_df, model, split_idxs, model_kwargs=hps, feature_cols=feature_cols, class_col=class_col)
    
    data_df,split_idxs = train_test_splits(df, class_balance=class_balance, class_col=class_col, test_size=test_size, n_splits=n_splits)
    train_metrics = eval_classifiers(data_df, classifiers, split_idxs, evaluator, eval_set='train',feature_cols=feature_cols, class_col=class_col)
    
    data_df,split_idxs = train_test_splits(df, class_balance=class_balance, class_col=class_col, test_size=test_size, n_splits=n_splits)
    test_metrics = eval_classifiers(data_df, classifiers, split_idxs, evaluator, eval_set='test',feature_cols=feature_cols, class_col=class_col)
    
    print(train_metrics,test_metrics)


In [ ]:

all_hps = {#'gamma': 0.1,
 'learning_rate': 4195.186967321277,
 'max_depth': int(2.0),
 'n_estimators': int(730.0),
 'reg_alpha': 0.3924498620840177,
 'reg_lambda': 1819330.6569741974}

feat_imps = []
for classifier in classifiers:
    feat_imps.append(classifier.feature_importances_)
feat_imps_arr = np.vstack(feat_imps)
mean_feat_imps = np.mean(feat_imps_arr,axis=0)
std_feat_imps = stats.sem(feat_imps_arr,axis=0)


In [ ]:
# double check the order!

importance = pd.DataFrame({'Behavior':feat_imps_arr[:,0],'ZT':feat_imps_arr[:,1],'SWA':feat_imps_arr[:,2],'FR':feat_imps_arr[:,3]
                          ,'CV':feat_imps_arr[:,4],'DCC':feat_imps_arr[:,5]})


In [55]:
importance.describe()

,Behavior,CT,SWA,FR,CV,DCC,Alpha,Beta
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.198631,0.123702,0.073256,0.107817,0.075365,0.261559,0.088350,0.071320
std,0.027977,0.017883,0.009692,0.012833,0.013411,0.037176,0.013596,0.011441
min,0.141823,0.083017,0.051023,0.077092,0.049186,0.174056,0.057322,0.043161
25%,0.178970,0.109846,0.066355,0.099115,0.066218,0.237331,0.077414,0.062967
50%,0.197899,0.121784,0.071959,0.108034,0.073501,0.257071,0.086739,0.070160
75%,0.215796,0.134286,0.079466,0.116185,0.083997,0.281693,0.096128,0.080246
max,0.275412,0.183010,0.106074,0.146431,0.132051,0.394391,0.130438,0.101976
